# Practical Exercise 6.01: Comparing fund performance with benchmark performance.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Sample data: Simulated daily returns of a fund and benchmark
np.random.seed(42)
dates = pd.date_range(start='2025-01-01', periods=252, freq='B')
fund_returns = np.random.normal(0.0005, 0.01, len(dates))  # Simulating daily fund returns
benchmark_returns = np.random.normal(0.0004, 0.009, len(dates))  # Simulating benchmark returns

# Convert to pandas DataFrame
data = pd.DataFrame({'Date': dates, 'Fund_Returns': fund_returns, 'Benchmark_Returns': benchmark_returns})
data.set_index('Date', inplace=True)

# Risk-free rate (annualized)
risk_free_rate = 0.02
trading_days = 252  # Annual trading days

# Function to calculate key metrics
def calculate_metrics(df, risk_free_rate):
    excess_returns = df['Fund_Returns'] - df['Benchmark_Returns']
    cumulative_return = (1 + df['Fund_Returns']).prod() - 1
    annualized_return = (1 + cumulative_return) ** (trading_days / len(df)) - 1
    benchmark_cumulative_return = (1 + df['Benchmark_Returns']).prod() - 1
    benchmark_annualized_return = (1 + benchmark_cumulative_return) ** (trading_days / len(df)) - 1
    annualized_volatility = df['Fund_Returns'].std() * np.sqrt(trading_days)
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility
    sortino_ratio = (annualized_return - risk_free_rate) / (df[df['Fund_Returns'] < 0]['Fund_Returns'].std() * np.sqrt(trading_days))
    max_drawdown = (df['Fund_Returns'].cumsum().expanding().max() - df['Fund_Returns'].cumsum()).max()
    tracking_error = np.std(excess_returns) * np.sqrt(trading_days)
    information_ratio = (annualized_return - df['Benchmark_Returns'].mean() * trading_days) / tracking_error
    beta = np.cov(df['Fund_Returns'], df['Benchmark_Returns'])[0,1] / np.var(df['Benchmark_Returns'])
    alpha = annualized_return - (risk_free_rate + beta * (df['Benchmark_Returns'].mean() * trading_days - risk_free_rate))
    calmar_ratio = annualized_return / abs(max_drawdown)

    return {
        'Annualized Return': annualized_return,
        'Benchmark Annualized Return': benchmark_annualized_return,
        'Annualized Volatility': annualized_volatility,
        'Sharpe Ratio': sharpe_ratio,
        'Sortino Ratio': sortino_ratio,
        'Maximum Drawdown': max_drawdown,
        'Tracking Error': tracking_error,
        'Information Ratio': information_ratio,
        'Beta': beta,
        'Alpha': alpha,
        'Calmar Ratio': calmar_ratio
    }

# Calculate metrics
metrics = calculate_metrics(data, risk_free_rate)

# Display results
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

# Plot Fund vs. Benchmark Returns Over Time
plt.figure(figsize=(10, 5))
plt.plot(data.index, data['Fund_Returns'].cumsum(), label='Fund', color='blue')
plt.plot(data.index, data['Benchmark_Returns'].cumsum(), label='Benchmark', color='orange')
plt.xlabel("Date")
plt.ylabel("Cumulative Returns")
plt.title("Comparison of Fund and Benchmark Cumulative Returns")
plt.legend()
plt.show()
